In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset


In [14]:
df = pd.read_csv(fr'C:\Users\ivane\Desktop\Dissertation\data\merged_data.csv', parse_dates=['Date'])
df.head()

,Date,Open,Close,Change,sentiment,sentiment_score
0,2025-01-10,3761.549,3736.886,-0.66,0,0.700804
1,2024-12-19,3780.106,3708.531,-1.93,0,0.950467
2,2024-12-18,3774.744,3781.551,0.18,1,0.460880
3,2024-12-16,3748.220,3765.163,0.45,1,0.771217
4,2024-12-12,3744.519,3748.267,0.10,1,0.927686


In [15]:
df.columns

Index(['Date', 'Open', 'Close', 'Change', 'sentiment', 'sentiment_score'], dtype='object')

In [18]:
features = ['Open', 'Close', 'Change', 'sentiment', 'sentiment_score']
target = 'Close'

In [19]:
scaler = MinMaxScaler()
df[features] = scaler.fit_transform(df[features])

In [ ]:
def create_sequences(data, target, window_size=10):
    